In [30]:
# Import Excel Data
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
df = pd.read_excel('Data_Train.xlsx')
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [32]:
df.shape

(10683, 11)

In [33]:
missing_values = df.isnull().sum()


In [34]:
# Drop missing values
df.dropna(inplace=True)

In [35]:
# Types of features
feature_types = df.dtypes
print("Types of Features:")
print(feature_types)


Types of Features:
Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object


In [36]:
# Convert 'Date_of_Journey' column to datetime
df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'])

# Find day and month
df['Day'] = df['Date_of_Journey'].dt.day
df['Month'] = df['Date_of_Journey'].dt.month


C:\Users\Asus\AppData\Local\Temp\ipykernel_17908\3745900265.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'])


In [37]:

# Drop duplicates based on day
df.drop_duplicates(subset='Day', keep='first', inplace=True)


In [38]:

# Extract hour and minute from 'Dep_Time' and 'Arrival_Time'
df['Dep_Hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
df['Dep_Minute'] = pd.to_datetime(df['Dep_Time']).dt.minute
df['Arrival_Hour'] = pd.to_datetime(df['Arrival_Time']).dt.hour
df['Arrival_Minute'] = pd.to_datetime(df['Arrival_Time']).dt.minute

# Drop 'Dep_Time' and 'Arrival_Time' columns
df.drop(columns=['Dep_Time', 'Arrival_Time'], inplace=True)

C:\Users\Asus\AppData\Local\Temp\ipykernel_17908\4032094675.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dep_Hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
C:\Users\Asus\AppData\Local\Temp\ipykernel_17908\4032094675.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dep_Minute'] = pd.to_datetime(df['Dep_Time']).dt.minute
C:\Users\Asus\AppData\Local\Temp\ipykernel_17908\4032094675.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Arrival_Hour'] = pd.to_datetime(df['Arrival_Time']).dt.hour
C:\Users\Asus\AppData\Local\Temp\ipykernel_17908\4032094675.py:5

In [39]:
# Drop 'Airline' and 'Destination' columns
df.drop(columns=['Airline', 'Destination'], inplace=True)

In [40]:
# Outlier detection
from sklearn.ensemble import IsolationForest

# Assuming only numerical columns are considered for outlier detection
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Initialize the model
outlier_detector = IsolationForest(contamination=0.1)

# Fit the model
outliers = outlier_detector.fit_predict(df[numerical_cols])

# Add outlier labels to the dataframe
df['Outlier'] = outliers

In [41]:

# Applying Regression Model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [42]:
# Split data into X and y
X = df.drop(columns=['Price'])
y = df['Price']

In [43]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [44]:
# Initialize the model
model = LinearRegression()

In [45]:
# Drop non-numeric columns
non_numeric_cols = X.select_dtypes(exclude=['int64', 'float64']).columns
X_train.drop(columns=non_numeric_cols, inplace=True)
X_test.drop(columns=non_numeric_cols, inplace=True)

# Initialize the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)


ValueError: at least one array or dtype is required

In [ ]:
# Predict
y_pred = model.predict(X_test)


In [ ]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
mse

66696242.5

In [ ]:
# Calculate R^2 Score
r2 = r2_score(y_test, y_pred)
r2

-0.9053362483382628

In [ ]:
# Adjested R2 score
X_test.shape
1 - ((1-r2)*(41-1)/(41-1-1))


-0.9541910239366798